# Demand Response in Grid2OP.
Try me out interactively with: [![Binder](./img/badge_logo.svg)](https://mybinder.org/v2/gh/Grid2Op/grid2op/master)
                                
**Objectives**

In the previous notebooks, we presented actions in a discrete action space. However, more actions are available in Grid2Op. Redispatching and curtailment are kind of continuous action that will be described here.

This notebook will:

- [I) Redispatching](#redispatching) details redispatching
    - present what redispatching is
    - show how it can be used in grid2op
    - detail the redispatching actions
    - show an example of a redispatching Agent
- [II) Curtailment](#curtailment) explains the curtailment
    - show the behaviour of the curtailment: how to perform it, what is does etc.
    - expose some limits of this curtailment: it can break the grid easily !


Execute the cell below by removing the # character if you use google colab !

<img src="https://colab.research.google.com/assets/colab-badge.svg" width="200">

Cell will look like:
```python
!pip install grid2op[optional]  # for use with google colab (grid2Op is not installed by default)
```


In [ ]:
# !pip install grid2op[optional]  # for use with google colab (grid2Op is not installed by default)

In [ ]:
import grid2op
from grid2op.Agent import DoNothingAgent, BaseAgent
from tqdm.notebook import tqdm  # for easy progress bar
from pathlib import Path
display_tqdm = False  # this is set to False for ease with the unitt test, feel free to set it to True
import numpy as np
max_iter = 100  # to make computation much faster we will only consider 100 time steps
import matplotlib.pyplot as plt

By default, some environments do not specify the cost of generators, their maximum and minimum production values, etc. In this case it is not possible to redispatch in grid2op.

To know more about what is needed for using redispatching, it is advised to look at this help online : https://grid2op.readthedocs.io/en/latest/space.html#grid2op.Space.GridObjects.redispatching_unit_commitment_availble for the most recent documentation. When this notebook was created, the following were needed:

- "load_size" : the maximum value a load can consume
- "load_flexible": whether this load is capable of providing demand response 
- "load_max_ramp_up": the maximum increase of power a generator can have between two consecutive time steps
- "load_max_ramp_down": the maximum decrease of power a generator can have between two consecutive time steps
- "load_cost_per_MW": the cost of activating 1MW of demand response. 

We made available a dedicated environment, based on the IEEE case5 powergrid that has all this features. It is advised to use this small environment for testing and getting familiar with demand response.

In [ ]:
try:
    from lightsim2grid import LightSimBackend
    bk_cls = LightSimBackend
except ImportError as exc:
    print(f"Error: {exc} when importing faster LightSimBackend")
    from grid2op.Backend import PandaPowerBackend
    bk_cls = PandaPowerBackend

env_name = "rte_case5_flexibility"
backend = bk_cls()
env = grid2op.make(env_name, test=True, backend=backend)

print(f"Is this environment suitable for redispatching: {env.redispatching_unit_commitment_availble}")
print(f"Is this environment suitable for flexibility: {env.load_flexibility_is_available}")

In [ ]:
env.load_size

In [ ]:
act = env.action_space({"flexibility": [(0, +1)]})
act.is_ambiguous()

In [ ]:
import matplotlib.pyplot as plt
from grid2op.PlotGrid import PlotMatplot
act = env.action_space({"flexibility": [(1, -1)],
                        "redispatch": [(1, 0.5)]
                        })
env.set_id(0)  # make sure to use the same environment input data.
obs_init = env.reset()
plotter = PlotMatplot(env.observation_space)
print(act)
obs, reward, done, info = env.step(act)
plotter.plot_obs(obs)
plt.show()
print(f"Target Dispatch: {obs.target_dispatch}")
print(f"Actual Dispatch: {obs.actual_dispatch}")
print(f"Target Flex: {obs.target_flex}")
print(f"Actual Flex: {obs.actual_flex}")
losses = np.abs(np.abs(obs.p_or)-np.abs(obs.p_ex)).sum()
print(f"Total Load: {obs.load_p.sum():.2f} :: Total Gen: {obs.gen_p.sum():.2f} :: Losses {losses:.2f}")
print(f"Balance: {obs.load_p.sum()+losses:.2f} == {obs.gen_p.sum():.2f}")

In [ ]:
# -1 Flex: 1.6     , 25.357508
# 0 Flex: 1.6     , 26.275875
obs.gen_p

In [ ]:
# -1 Flex: 8.8     , 7.      , 8.230771
# 0 Flex: 8.8, 8. , 8.
obs.load_p

In [ ]:
from grid2op.Parameters import Parameters
p = Parameters()
p.NB_TIMESTEP_OVERFLOW_ALLOWED = 1
p.MAX_LINE_STATUS_CHANGED = 17
p.MAX_SUB_CHANGED = 14
env = grid2op.make("cigre_enhanced_large_train", allow_detachment=True, param=p)

obs = env.reset(options={"time serie id": "Scenario_2"})


In [ ]:
import matplotlib.pyplot as plt
from grid2op.PlotGrid import PlotMatplot

obs = env.reset(options={"time serie id": "Scenario_3068"})

plotter = PlotMatplot(env.observation_space, load_name=True, gen_name=True)

LOAD = "Load CI7"
act = env.action_space({"flexibility":[(LOAD, obs.load_max_ramp_up[np.nonzero(obs.name_load == LOAD)[0]])],
                        "curtail": [("gen_7_8", 0.0)]})
obs, reward, done, info = env.step(act)
plotter.plot_obs(obs)
plt.show()

losses = np.abs(np.abs(obs.p_or)-np.abs(obs.p_ex)).sum()
print(f"Total Load: {obs.load_p.sum():.2f} :: Total Gen: {obs.gen_p.sum():.2f} :: Losses {losses:.2f}")
print(f"Balance: {obs.load_p.sum()+losses:.2f} == {obs.gen_p.sum():.2f}")